<a href="https://colab.research.google.com/github/karthikram78/prodigyinfotech-/blob/main/prodigyinfotech%20task%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch, platform, sys, os, subprocess, textwrap, random
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA available: False
Device name: CPU


In [ ]:
!pip -q install --upgrade kagglehub gradio torchvision torchmetrics timm pandas pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 107.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


In [ ]:
import os, glob, tarfile, zipfile, kagglehub, shutil

# Try to download with kagglehub (no manual kaggle.json needed in most cases)
food_root = kagglehub.dataset_download("dansbecker/food-101")  # returns a cache path
print("KaggleHub path:", food_root)

# Find or extract the actual 'food-101' directory that contains images/ and meta/
def ensure_food101_extracted(base):
    # If already extracted
    for cand in [base] + [os.path.join(base, n) for n in os.listdir(base)]:
        if os.path.isdir(os.path.join(cand, "food-101")):
            if os.path.isdir(os.path.join(cand, "food-101", "images")) and os.path.isdir(os.path.join(cand, "food-101", "meta")):
                return os.path.join(cand, "food-101")
        if os.path.isdir(os.path.join(cand, "images")) and os.path.isdir(os.path.join(cand, "meta")):
            return cand

    # Look for archives to extract
    archives = []
    for ext in ("*.zip","*.tar.gz","*.tgz"):
        archives += glob.glob(os.path.join(base, "**", ext), recursive=True)
    # Prefer the big food-101 archive if present
    archives = sorted(archives, key=lambda p: (not ("food-101" in os.path.basename(p).lower()), len(p)))

    extract_dir = os.path.join(base, "_extracted")
    os.makedirs(extract_dir, exist_ok=True)
    for arc in archives:
        print("Extracting:", arc)
        try:
            if arc.endswith(".zip"):
                with zipfile.ZipFile(arc, 'r') as zf:
                    zf.extractall(extract_dir)
            else:
                with tarfile.open(arc, 'r:*') as tf:
                    tf.extractall(extract_dir)
        except Exception as e:
            print("Extraction failed for", arc, "->", e)

    # Re-scan for the structure
    for cand in [extract_dir] + [os.path.join(extract_dir, n) for n in os.listdir(extract_dir)]:
        if os.path.isdir(os.path.join(cand, "food-101", "images")) and os.path.isdir(os.path.join(cand, "food-101", "meta")):
            return os.path.join(cand, "food-101")
        if os.path.isdir(os.path.join(cand, "images")) and os.path.isdir(os.path.join(cand, "meta")):
            return cand

    raise RuntimeError("Could not find/extract Food-101 with images/ and meta/ folders.")

food101_dir = ensure_food101_extracted(food_root)
print("Food-101 directory:", food101_dir)


100%|██████████| 9.38G/9.38G [01:35<00:00, 105MB/s]

Extracting files...


KaggleHub path: /root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1
Food-101 directory: /root/.cache/kagglehub/datasets/dansbecker/food-101/versions/1/food-101/food-101


In [ ]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision

meta_dir = os.path.join(food101_dir, "meta")
images_dir = os.path.join(food101_dir, "images")

# classes.txt provides canonical class order
with open(os.path.join(meta_dir, "classes.txt")) as f:
    classes = [line.strip() for line in f]
class_to_idx = {c:i for i,c in enumerate(classes)}
num_classes = len(classes)
print("Classes:", len(classes), "examples:", classes[:8])

# Read official splits (lines are like 'apple_pie/1005649')
def read_split(list_file):
    items = []
    with open(list_file) as f:
        for line in f:
            rel = line.strip() + ".jpg"
            cls = rel.split("/")[0]
            items.append((os.path.join(images_dir, rel), class_to_idx[cls]))
    return items

train_items = read_split(os.path.join(meta_dir, "train.txt"))
test_items  = read_split(os.path.join(meta_dir, "test.txt"))
print("Train images:", len(train_items), " Test images:", len(test_items))

# Quick demo mode (train faster on a subset). Set to False for full training.
QUICK_TRAIN = True
if QUICK_TRAIN:
    import random
    random.seed(0)
    keep_classes = set(random.sample(classes, 20))       # 20 classes for speed
    train_items = [s for s in train_items if classes[s[1]] in keep_classes][:20*500]  # ~10k images
    test_items  = [s for s in test_items  if classes[s[1]] in keep_classes][:20*100]  # ~2k images
    classes = sorted(list(keep_classes))
    class_to_idx = {c:i for i,c in enumerate(classes)}
    # reindex labels
    def remap(items):
        out=[]
        for p, y in items:
            c = os.path.basename(os.path.dirname(p))
            out.append((p, class_to_idx[c]))
        return out
    train_items, test_items = remap(train_items), remap(test_items)
    num_classes = len(classes)
    print("QUICK_TRAIN on", num_classes, "classes. Train:", len(train_items), "Test:", len(test_items))

# Transforms (use ImageNet stats; EfficientNet_B0 expects 224x224)
weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
val_tf = weights.transforms()
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=weights.meta["mean"], std=weights.meta["std"]),
])

class FoodListDataset(Dataset):
    def __init__(self, items, transform):
        self.items = items
        self.transform = transform
    def __len__(self): return len(self.items)
    def __getitem__(self, idx):
        path, y = self.items[idx]
        img = Image.open(path).convert("RGB")
        img = self.transform(img)
        return img, y

train_ds = FoodListDataset(train_items, train_tf)
test_ds  = FoodListDataset(test_items,  val_tf)

BATCH = 32
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)


Classes: 101 examples: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap']
Train images: 75750  Test images: 25250
QUICK_TRAIN on 20 classes. Train: 10000 Test: 2000


KeyError: 'mean'

In [ ]:
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=weights.meta["mean"], std=weights.meta["std"]),
])


KeyError: 'mean'

In [ ]:
from torchvision import transforms

# Try to get default preprocessing from the weights
try:
    # Newer torchvision (>=0.14) – transforms is a callable
    val_tf = weights.transforms()
except TypeError:
    # Older torchvision – transforms is already a Compose
    val_tf = weights.transforms

# For training, add augmentation on top of the eval transforms
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    # Append the rest of val_tf (ToTensor + Normalize)
    *val_tf.transforms[-2:]
])


AttributeError: 'ImageClassification' object has no attribute 'transforms'

In [ ]:
from torchvision import transforms

# Standard ImageNet normalization (used for all pretrained models)
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

# Validation transforms (no augmentation)
val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

# Training transforms (augmentation + same normalization)
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])


In [ ]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import random, os

meta_dir = os.path.join(food101_dir, "meta")
images_dir = os.path.join(food101_dir, "images")

# classes.txt provides canonical class order
with open(os.path.join(meta_dir, "classes.txt")) as f:
    classes = [line.strip() for line in f]
class_to_idx = {c: i for i, c in enumerate(classes)}
num_classes = len(classes)
print("Classes:", len(classes), "examples:", classes[:8])

# Read official splits
def read_split(list_file):
    items = []
    with open(list_file) as f:
        for line in f:
            rel = line.strip() + ".jpg"
            cls = rel.split("/")[0]
            items.append((os.path.join(images_dir, rel), class_to_idx[cls]))
    return items

train_items = read_split(os.path.join(meta_dir, "train.txt"))
test_items  = read_split(os.path.join(meta_dir, "test.txt"))
print("Train images:", len(train_items), " Test images:", len(test_items))

# Quick demo mode (train faster on subset). Set to False for full training.
QUICK_TRAIN = True
if QUICK_TRAIN:
    random.seed(0)
    keep_classes = set(random.sample(classes, 20))       # 20 classes for speed
    train_items = [s for s in train_items if classes[s[1]] in keep_classes][:20*500]  # ~10k images
    test_items  = [s for s in test_items  if classes[s[1]] in keep_classes][:20*100]  # ~2k images
    classes = sorted(list(keep_classes))
    class_to_idx = {c:i for i,c in enumerate(classes)}
    # reindex labels
    def remap(items):
        out=[]
        for p, y in items:
            c = os.path.basename(os.path.dirname(p))
            out.append((p, class_to_idx[c]))
        return out
    train_items, test_items = remap(train_items), remap(test_items)
    num_classes = len(classes)
    print("QUICK_TRAIN on", num_classes, "classes. Train:", len(train_items), "Test:", len(test_items))

# ✅ FIXED transforms (hardcoded ImageNet normalization)
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

class FoodListDataset(Dataset):
    def __init__(self, items, transform):
        self.items = items
        self.transform = transform
    def __len__(self): return len(self.items)
    def __getitem__(self, idx):
        path, y = self.items[idx]
        img = Image.open(path).convert("RGB")
        img = self.transform(img)
        return img, y

train_ds = FoodListDataset(train_items, train_tf)
test_ds  = FoodListDataset(test_items,  val_tf)

BATCH = 32
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)


Classes: 101 examples: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap']
Train images: 75750  Test images: 25250
QUICK_TRAIN on 20 classes. Train: 10000 Test: 2000


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets

# 🔹 Dataloaders
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_tf)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_tf)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

print(f"Train images: {len(train_dataset)} | Test images: {len(val_dataset)}")
print(f"Classes: {train_dataset.classes[:8]} ... total {len(train_dataset.classes)} classes")

# 🔹 Model (example: ResNet18 with pretrained weights)
from torchvision.models import resnet18, ResNet18_Weights

weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
model.fc = torch.nn.Linear(model.fc.in_features, len(train_dataset.classes))  # change output layer

# 🔹 Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 🔹 Loss & Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


NameError: name 'train_dir' is not defined

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ✅ Path to Food-101 dataset
dataset_path = "/content/food-101/images"

# ✅ Create train/val split (80-20)
all_classes = os.listdir(dataset_path)
train_dir = "/content/food-101/train"
val_dir = "/content/food-101/val"

# Create folders if not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for cls in all_classes:
    cls_path = os.path.join(dataset_path, cls)
    images = os.listdir(cls_path)

    # Split into 80% train, 20% validation
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))
    for img in val_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(val_dir, cls, img))

print("✅ Dataset split into train and validation!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/food-101/images'

In [ ]:
# ✅ Train/Val directories
train_dir = "/content/food-101/train"
val_dir = "/content/food-101/val"

# Create split if not already done
if not os.path.exists(train_dir):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    import shutil
    from sklearn.model_selection import train_test_split

    all_classes = os.listdir(os.path.join(dataset_path, "images"))
    for cls in all_classes:
        cls_path = os.path.join(dataset_path, "images", cls)
        images = os.listdir(cls_path)

        train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

        for img in train_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))

        for img in val_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(val_dir, cls, img))

print("✅ Train/Val split ready!")



FileNotFoundError: [Errno 2] No such file or directory: '/content/food-101/images/images'

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# ✅ Path to Food-101 dataset
dataset_path = food101_dir  # ✅ Use the path from previous extraction
images_dir = os.path.join(dataset_path, "images") # ✅ Construct the path to the images folder

train_dir = "/content/food-101/train"
val_dir = "/content/food-101/val"

# Create train and val folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get all classes, ignoring non-directory entries like .DS_Store
all_classes = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))] # ✅ Use the correct path and filter

for cls in all_classes:
    cls_path = os.path.join(images_dir, cls) # ✅ Use the correct path
    images = os.listdir(cls_path)

    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))
    for img in val_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(val_dir, cls, img))

print("✅ Train/Val split created successfully!")

✅ Train/Val split created successfully!


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

# ✅ Define transforms (using the ones defined previously)
# Assuming train_tf and val_tf are defined in a previous cell
# If not, you might need to redefine them here or make sure the cell
# where they are defined has been run.

# ✅ Load dataset from the created train/val directories
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_tf)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_tf)

# ✅ Dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH, shuffle=False, num_workers=2)

print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")
print(f"Classes: {train_dataset.classes[:8]} ... total {len(train_dataset.classes)} classes")

Training images: 80800
Validation images: 20200
Classes: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap'] ... total 101 classes


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# ✅ Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ✅ Load pretrained ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# ✅ Modify final layer to match number of food classes
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

# ✅ Loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)


Using device: cpu
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReL

In [ ]:
import torch.nn as nn
import torchvision.models as models

# ✅ Load a pretrained ResNet model
model = models.resnet18(pretrained=True)

# ✅ Freeze the feature extractor (optional, speeds up training)
for param in model.parameters():
    param.requires_grad = False

# ✅ Replace the final layer to match 101 food classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_dataset.classes))

# ✅ Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(model)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch.optim as optim

# ✅ Loss function
criterion = nn.CrossEntropyLoss()

# ✅ Optimizer (only train the final layer)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# ✅ Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

print("Loss, Optimizer, and Scheduler are ready ✅")


Loss, Optimizer, and Scheduler are ready ✅


In [ ]:
import time

EPOCHS = 10  # you can increase later

best_acc = 0.0
start_time = time.time()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("-" * 30)

    # ---- Training ----
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    print(f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    val_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

    val_loss = val_loss / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)

    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # ---- Save best model ----
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("✅ Best model saved!")

    scheduler.step()

total_time = time.time() - start_time
print(f"\nTraining complete in {total_time//60:.0f}m {total_time%60:.0f}s")
print(f"Best Val Accuracy: {best_acc:.4f}")



Epoch 1/10
------------------------------


KeyboardInterrupt: 